# Data cleaning - Part 1

Il webscraper ha sbagliato principalmente 2 cose:
- in alcune pagine non era indicata la tipologia della casa in un apposito campo, ma solo la classe catastale, 'A/2' e 'A/7' nei casi osservati, che corrispondono alle Ville e ai Villini
- in altre pagine la condizione dell'appartamento appariva nel campo solitamente destinato all'anno

In questo notebook eliminiamo i due fenomeni e creiamo un nuovo .csv "pulito".

In [1]:
import numpy as np
import pandas as pd

with open('immobiliare_data_raw.csv', 'r') as data_file:
    data = pd.read_csv(data_file)

In [12]:
types_list = [
    'Appartamento',
    'Attico',
    'Loft',
    'Open Space',
    'Mansarda',
    'Soffitta'
]

condition_list = [
    'Nuovo / In costruzione',
    'Nuovo/In costruzione'
    'Da ristrutturare',
    'Da ristrutturare',
    'Ottimo/Ristrutturato',
    'Ottimo / Ristrutturato',
    'Buono / Abitabile',
    'Buono/Abitabile'
]

wrong_apt_idxs = []
wrong_vil_idxs = []
wrong_year_idxs = []
for k,row in data.iterrows():
    if (
        isinstance(row.condition, str) 
        and row.condition.find('Classe A/2') > -1
        and row.type not in types_list
    ):
        wrong_apt_idxs.append(k)
        
    if (
        isinstance(row.condition, str) 
        and row.condition.find('Classe A/7') > -1
        and row.type not in types_list
    ):
        wrong_vil_idxs.append(k)
        
    if (
        isinstance(row.condition, str) and isinstance(row.year_built, str)
        and all([t < 0 for t in [row.condition.find(cond) for cond in condition_list]])
        and any([s > -1 for s in [row.year_built.find(cond) for cond in condition_list]]) 
    ):
        wrong_year_idxs.append(k)
        
print(len(wrong_apt_idxs), len(wrong_vil_idxs), len(wrong_year_idxs))

21 51 7876


In [13]:
data.loc[wrong_apt_idxs,'type'] = 'Villa'
data.loc[wrong_apt_idxs,'condition'] = 'NA'

data.loc[wrong_vil_idxs,'type'] = 'Villino'
data.loc[wrong_vil_idxs,'condition'] = 'NA'

data.loc[wrong_year_idxs,'condition'] = data.loc[wrong_year_idxs,'year_built']
data.loc[wrong_year_idxs,'year_built'] = '0'

with open('immobiliare_data.csv', 'w') as out_file:
    data.to_csv(out_file)